In [150]:
import pandas as pd

def load_some_data(path, separ = ','):
    return pd.read_csv(path, sep=separ, parse_dates = ['datetime'], index_col='datetime')

events_battle = load_some_data(r'data/events_battle.txt')
events_hero = load_some_data(r'data/events_hero.txt')
events_level = load_some_data(r'data/events_level.txt', '\t')
events_payment = pd.read_csv(r'data/events_payment.txt', parse_dates = ['created'], index_col='created')
events_quest = load_some_data(r'data/events_quest.txt')
events_resource = load_some_data(r'data/events_resource.txt')
events_user = load_some_data(r'data/events_user.txt')

In [44]:

def get_relation_on_battles(ds1, ds2):
    t1 = ds1[['user_id', 'status']].groupby('user_id').agg('count')
    t2 = ds2[['user_id', 'status']].groupby('user_id').agg('count')
    return t1 / t2


In [112]:
#объявление всех сетов для анализа
no_bot = events_battle.loc[events_battle['opponent_type'] != 'bot']
duel = events_battle.loc[events_battle['kind'] =='duel']
only_bot = events_battle.loc[events_battle['opponent_type'] == 'bot']
duels_with_bots = events_battle.loc[(events_battle['opponent_type'] == 'bot') & (events_battle['kind'] =='duel')] #zero
duels_with_supers = duels_with_bots = events_battle.loc[(events_battle['opponent_race_id'] == 'super-hero') & (events_battle['kind'] =='duel')]#zero

In [49]:
#самая популярная раса
#если что, то superhero - раса по умолчанию
events_hero.copy()[["user_id", "race"]].groupby("race").agg('count')

,user_id
race,
elf,1626
human,1472
orc,1221
super-hero,6075


In [48]:
#самая играбельная раса для каждого игрока
def get_most_race(ds):
    cur_race = pd.merge(ds, events_hero, on = 'hero_id')
    return cur_race.groupby("user_id_x")['race'].agg(lambda x: pd.Series.mode(x)[0]).to_frame()


In [93]:
#распределение винрейтов расс в датасете
def get_race_winrate_disturb(df):
    tmp = pd.merge(df, events_hero, on = 'hero_id')
    tmp = tmp[['race', 'status']]
    win = tmp.reset_index().loc[tmp['status'] == 'win'].groupby('race')[['status']].agg('count')
    win.fillna(0, inplace = True)
    tmp = tmp.reset_index().groupby('race')[['status']].agg('count')
    return (win/tmp).fillna(0)
    

In [91]:
def get_players_winrates(df):
    win = df.loc[df['status'] == 'win'][['user_id','status']].groupby('user_id').agg('count')
    tot = df[['user_id','status']].groupby('user_id').agg('count')
    return (win/tot).fillna(0)

In [92]:
get_players_winrates(duel)

,status
user_id,
001a3472-44bc-4583-95d9-bf8759085017,0.000000
002bcd7d-83b1-42ac-b257-ed71a4585f5e,0.500000
00a21109-ed6d-4be2-8e20-b79158da28b8,0.500000
00c7d924-83f5-4679-9d96-35356af1f53d,0.818182
00dd3007-71cf-42de-9c46-680988d89242,0.000000
010a6e53-245a-47a3-8e44-c8212c101399,0.000000
0124930f-142b-4135-95b7-1085e44dcc63,0.375000
013a9402-6935-4c62-8da7-6463f8a03877,0.000000
01426a80-0ccb-4c2f-b3a5-1db1ab2343a8,0.000000


In [113]:
duels_with_supers

,id,transaction_id,user_id,hero_id,status,duration,ai_share,experience,crystals,mana,location_id,opponent_type,opponent_user_id,opponent_hero_id,bot_template_id,opponent_race_id,kind
datetime,,,,,,,,,,,,,,,,,


In [132]:
#import re
#pm = re.compile(r'.*(dog)$')
#only_bot = events_hero.loc[events_hero['user_id'].str.contains(r'super')]
#only_bot
events_hero.groupby('user_id').agg('count').sort_values(by = 'hero_id') 
events_hero.loc[events_hero['hero_id'] == 'super-hero'] # супергероя никто не создал, ибо это дефолт

,id,transaction_id,user_id,hero_id,race,name
datetime,,,,,,


In [154]:
for_sub = events_hero.groupby('user_id').agg('count')[['id']]
for_sub['id'] = 
def_users = events_user.set_index('user_id')[''].subtract(events_hero.groupby('user_id').agg('count'))
#def_users.describe() 
events_hero.groupby('user_id').agg('count')
#def_users_with_name = def_users.loc[pd.notna(def_users['name'])]
#def_users_with_name # узнаём, что все люди с одним персонажем не имеют ника -> все, кто играет под своим ником, пилили второго перса

,id,transaction_id,hero_id,race,name
user_id,,,,,
00040422-b869-4272-ba05-1cc9481037c9,2,2,2,2,2
000f77b1-660b-4cf2-87f0-d4f3b3f9a718,1,1,1,1,1
001a3472-44bc-4583-95d9-bf8759085017,2,2,2,2,2
0026badb-5654-4c8d-956c-e0d3cb7a3e6b,1,1,1,1,1
002bcd7d-83b1-42ac-b257-ed71a4585f5e,2,2,2,2,2
0030c0b3-9683-41c0-bcac-b2aad03759f4,2,2,2,2,2
003c3099-4382-4f86-a989-a847628894f0,2,2,2,2,2
00467ba2-b71c-4a69-b8bc-d7491cf2eca0,2,2,2,2,2
0047e219-202a-4eda-8d31-b35a58f50eec,3,3,3,3,3


In [152]:
events_user.head()

,id,user_id,region_id
datetime,,,
2018-10-13 03:00:36,189,de8b8cff-8ff2-4737-8421-5cfb6e6d858d,ru
2018-10-13 03:03:12,190,1a674288-28d0-495b-8c21-8c54f06088f6,ru
2018-10-13 03:07:57,191,33b8bddf-a67d-4ceb-a63d-0b077c711c96,en
2018-10-13 03:35:41,192,f6bfd30b-dbd2-498c-b222-257936cd01f3,ru
2018-10-13 03:46:18,193,40f0153b-2b89-4e92-91b1-9b3eead3fe62,ru
